In [1]:
import pickle
import torch
from torch import nn
import numpy as np
import linecache
import re
import os
import pandas as pd
import linecache
import time
import math 

In [2]:
summary_df = pd.read_csv('/Users/Jacob_Diaz/Desktop/Cornell/Fall 2022/CS 4775/final_project/data/summary.csv')
summary_df.head()

,Unnamed: 0,ID,antibody,cell,composite,control,controlId,dataType,dataVersion,dccAccession,lab,md5sum,project,quality,size,tableName,treatment,type,view
0,0,wgEncodeAwgTfbsBroadDnd41CtcfUniPk,CTCF,Dnd41,wgEncodeAwgTfbsUniform,std,wgEncodeEH002434,ChipSeq,ENCODEMar2012Freeze,wgEncodeEH002402,Broad,d24dcb9899cf3f58dfb83e8701b9e9cc,wgEncode,good,966K,wgEncodeAwgTfbsBroadDnd41CtcfUniPk,None,narrowPeak,Peaks
1,1,wgEncodeAwgTfbsBroadDnd41Ezh239875UniPk,EZH2_(39875),Dnd41,wgEncodeAwgTfbsUniform,std,wgEncodeEH002434,ChipSeq,ENCODEMar2012Freeze,wgEncodeEH003080,Broad,effd48196e39d321ad9f4e2c857dbe88,wgEncode,caution,43K,wgEncodeAwgTfbsBroadDnd41Ezh239875UniPk,None,narrowPeak,Peaks
2,2,wgEncodeAwgTfbsBroadGm12878CtcfUniPk,CTCF,GM12878,wgEncodeAwgTfbsUniform,std,wgEncodeEH000037,ChipSeq,ENCODEMar2012Freeze,wgEncodeEH000029,Broad,fef0af7fe1e724159e665085da53efb1,wgEncode,good,982K,wgEncodeAwgTfbsBroadGm12878CtcfUniPk,None,narrowPeak,Peaks
3,3,wgEncodeAwgTfbsBroadGm12878Ezh239875UniPk,EZH2_(39875),GM12878,wgEncodeAwgTfbsUniform,std,wgEncodeEH000037,ChipSeq,ENCODEMar2012Freeze,wgEncodeEH002411,Broad,9a87661953cb6419847abd00599f0a30,wgEncode,good,66K,wgEncodeAwgTfbsBroadGm12878Ezh239875UniPk,None,narrowPeak,Peaks
4,4,wgEncodeAwgTfbsBroadH1hescChd1a301218aUniPk,CHD1_(A301-218A),H1-hESC,wgEncodeAwgTfbsUniform,std,wgEncodeEH000088,ChipSeq,ENCODEMar2012Freeze,wgEncodeEH002095,Broad,71891a536ed23a626f37d0c9b64ff2b1,wgEncode,good,184K,wgEncodeAwgTfbsBroadH1hescChd1a301218aUniPk,None,narrowPeak,Peaks


In [3]:
summary_df = summary_df[summary_df['quality'] == 'good']

In [4]:
summary_df['quality'].value_counts()

good    611
Name: quality, dtype: int64

In [5]:
summary_df['antibody'].value_counts()

CTCF               88
Pol2               48
c-Myc              17
NRSF               12
Pol2-4H8           12
                   ..
HNF4A_(SC-8987)     1
FOXA2_(SC-6554)     1
CEBPD_(SC-636)      1
SP4_(V-20)          1
eGFP-JunD           1
Name: antibody, Length: 172, dtype: int64

In [6]:
summary_df['cell'].value_counts()

K562       131
GM12878     74
HepG2       63
H1-hESC     55
HeLa-S3     51
          ... 
BJ           1
Caco-2       1
GM06990      1
GM12801      1
WI-38        1
Name: cell, Length: 88, dtype: int64

In [7]:
PATH = '/Users/Jacob_Diaz/Desktop/Cornell/Fall 2022/CS 4775/final_project/data'
folder_to_keys = pickle.load(open(f'{PATH}/folder_to_keys.pickle', 'rb'))

In [8]:
print(os.getcwd())

/Users/Jacob_Diaz/Desktop/Cornell/Fall 2022/CS 4775/final_project/notebooks


In [9]:
folder_to_keys['wgEncodeAwgTfbsBroadDnd41CtcfUniPk']['train']

{'chr12:31391876-31391976',
 'chr19:1590812-1590912',
 'chr7:15365799-15365899_shuf',
 'chr21:40525461-40525561_shuf',
 'chr4:115101836-115101936',
 'chr4:130448098-130448198',
 'chr18:48725772-48725872',
 'chr4:113269807-113269907',
 'chr8:105430400-105430500',
 'chr16:88038891-88038991',
 'chr19:39881430-39881530',
 'chr11:63683962-63684062_shuf',
 'chr2:134991019-134991119_shuf',
 'chr17:62067897-62067997',
 'chr6:56756714-56756814_shuf',
 'chr10:44167206-44167306_shuf',
 'chr8:60197777-60197877',
 'chr2:233924439-233924539',
 'chr11:57435508-57435608',
 'chr17:10782590-10782690_shuf',
 'chr22:23219782-23219882',
 'chr3:127192196-127192296',
 'chr22:20072581-20072681',
 'chr7:99787778-99787878',
 'chr12:13361541-13361641_shuf',
 'chr12:48207009-48207109_shuf',
 'chr21:32470967-32471067',
 'chr22:43486338-43486438',
 'chr16:72881180-72881280_shuf',
 'chr1:86965819-86965919',
 'chr17:34452857-34452957_shuf',
 'chr12:131072435-131072535',
 'chr6:11655931-11656031_shuf',
 'chr2:43819633

In [10]:
class DNADataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'

    def __init__(self, folder, set_type):
        'Initialization'
        self.folder = folder
        self.set_type = set_type  # Either 'train' or 'test'
        self.list_IDs = list(folder_to_keys[folder][set_type])

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
#         line = linecache.getline(
#             f'{PATH}/raw-numerical/{self.folder}/{self.set_type}.txt', index+1)
        line = linecache.getline(
            f'{PATH}/raw/{self.folder}/{self.set_type}.txt', index+1)
        result = re.search('(.*) (\w+) (\d+)', line)
        key = result[1]
        X = list(result[2])
#         X = result[2]
        y = result[3]

        return X, y
        


In [14]:
def get_k_mer_embedding(k, path):    
    k_mer_embedding = dict()
    with open(path) as f:
       for line in f:
            result = re.search('(\w*)(( (-?\d*\.?\d*))*)', line)
            curr_key = result[1]
            if len(result[1]) == k:
                vector = result[2].split(' ')
                vector = list(map(lambda x: float(x), vector[1:]))
                k_mer_embedding[curr_key] = np.array(vector)
        
    idx_to_kmer = list(k_mer_embedding.keys())
    kmer_to_idx = dict(zip(idx_to_kmer, np.arange(4**k)))
    embedding = np.array(list(k_mer_embedding.values()))
        
    assert(4**k == len(k_mer_to_idx))
    assert(embedding.shape[0] == 4**k)
    return kmer_to_idx, embedding, idx_to_kmer
                              
# PATH = f'/Users/Jacob_Diaz/Desktop/Cornell/Fall 2022/CS 4775/final_project/src/features/dna2vec/' 
# filepath = PATH +'pretrained/dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.txt'
x = get_k_mer_embedding(3, filepath)   

kmer_to_idx, embedding, idx_to_kmer = x[0], x[1], x[2]
# print(kmer_to_idx)                     
# print(embedding)  
# print(idx_to_kmer)

# Takes in a sequence of length 101 and returns a tensor
# containing 101-k+1 sliding window subsequences of length k

def old_seq_to_tensor(sequence, k):
    lst = []
    for i in range(len(sequence)-k+1):
        kmer = sequence[i:i+3]
        kmer_idx = k_mer_to_idx[kmer]
        lst.append(embedding[kmer_idx])
        
    return torch.tensor(lst)

def seq_to_tensor(sequence, k):
    lst = []
    kmers = np.lib.stride_tricks.sliding_window_view(sequence, k)
    for i in range(len(kmers)):
        kmer_lst = kmers[i]
        kmer = f'{kmer_lst[0]}{kmer_lst[1]}{kmer_lst[2]}'
        kmer_idx = kmer_to_idx[kmer]
        lst.append(kmer_idx)

    return np.array(lst)

def get_tensor(input_data, k):
    
    matrix = np.array(input_data).T
#     assert matrix.shape == (_,101), f'matrix.shape {matrix.shape} != (_,101)'
    
    return torch.tensor(np.apply_along_axis(seq_to_tensor, 1, matrix, k=k))
    

NameError: name 'filepath' is not defined

In [ ]:
g = np.array([['A','T','G','C'],['A','T','G','C'],['A','T','G','C']])

# data_to_tensor(g,3)

get_tensor(g,3)


# seq_to_tensor2(g[0], 3)

In [ ]:
def PaperEmbedding(k):
    assert(k > 2 and k < 9)
    # DNA 2 Vec Embedding
    PATH = f'/Users/Jacob_Diaz/Desktop/Cornell/Fall 2022/CS 4775/final_project/src/features/dna2vec/' 
    filepath = PATH +'pretrained/dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.txt'
    kmer_to_idx, embedding, idx_to_kmer = get_k_mer_embedding(k, filepath)
    dna2vec = torch.FloatTensor(embedding)

    # One Hot Embedding
    one_hot = torch.diag(torch.ones(4**k))
    
    #Concatination of the two
    embedding = nn.Embedding.from_pretrained(torch.cat( (dna2vec,one_hot) , axis = 1))
    
    return embedding


In [ ]:
class Convolution(nn.Module):
    def __init__(self, k):
        super(Convolution, self).__init__()
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5,5), padding=2)
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5,5), padding=2)
        
        
    # input_matrix is a b x (100-k+1) x (len(dna2vec dim) + len(positional encoding))
    def forward(self, input_matrix):
        
        input_reshaped = input_matrix[:,None,:,:]
        X = self.relu(self.conv1(input_reshaped))
        X = self.relu(self.conv2(X))
#         Paper does not explicitly softmax at the end
#         X = self.softmax(X)

        return X

In [ ]:
class LstmAttn(nn.Module):
    def __init__(self, k, input_dim, output_dim):
        super(LstmAttn, self).__init__()
        self.k = k
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.lstm = nn.LSTM(input_dim, output_dim, batch_first = True, bidirectional = True)
        
        self.w = nn.Linear(2*output_dim, 1)
        
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim = 1)
#         self.softmax = nn.LogSoftmax(dim = 1)
        
    def forward(self, input_matrix):
        
        #LSTM
        L = 101 - self.k + 1
#         assert( input_matrix.shape == ( 8, 1, L, self.input_dim) )
        
        lstm_input = torch.squeeze(input_matrix, 1)
        
        all_hiddens, (last_hidden, last_cell_state) =  self.lstm(lstm_input)
        H = all_hiddens
#         H = torch.permute(all_hiddens, ())
#         assert(H.shape == (8, L, 2*self.output_dim))

        # Attention
        M = self.tanh(H)
        wt_M = self.w(M)
#         assert(wt_M.shape == (8, L, 1)), f'wt_M.shape = {wt_M.shape}'
        
#         wt_M = torch.squeeze(wt_M, 2)
#         assert(wt_M.shape == (8, L)), f'wt_M.shape = {wt_M.shape}'
#         alpha = self.softmax(wt_M)
#         assert(alpha.shape == (8, L)), f'wt_M.shape = {wt_M.shape}'

        alpha = self.softmax(wt_M)
#         assert(alpha.shape == (8, L, 1)), f'wt_M.shape = {alpha.shape}'

        H = torch.permute(H, (0,2,1))
#         assert(H.shape == (8, 2*self.output_dim, L))
        
        r = torch.bmm(H, alpha)
#         assert(r.shape == (8, 2*self.output_dim, 1)), f'wt_M.shape = {r.shape}'

        h_star = self.tanh(r)
#         assert(h_star.shape == (8, 2*self.output_dim, 1)), f'wt_M.shape = {h_star.shape}'
        
        h_star = torch.squeeze(h_star, 2)
#         assert(h_star.shape == (8, 2*self.output_dim)), f'wt_M.shape = {h_star.shape}'
        
#         print(h_star)
        
        return h_star

In [ ]:
class BinaryClassificationHead(nn.Module):
    def __init__(self, k, input_dim, hidden_dim):
        super(BinaryClassificationHead, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = 2
        
        self.hl_1 = nn.Linear(input_dim, hidden_dim)
        self.hl_2 = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(p=0.2)
        
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, input_matrix):
#         assert(input_matrix.shape == (8, self.input_dim))
        
        # "back of the full connection layer"
        dropout = self.dropout(input_matrix)
        layer1 = self.hl_1(dropout)
        layer2 = self.hl_2(layer1)
        pred = self.sigmoid(layer2)
        
#         # "added between the fully connected layers"
#         layer1 = self.hl_1(input_matrix)
#         dropout = self.dropout(layer1)
#         layer2 = self.hl_2(dropout)
#         pred = self.sigmoid(layer2)
        
#         assert( pred.shape == (8, 1) )
        pred = torch.squeeze(pred, 1)

#         assert( pred.shape == (8,) ), f'pred.shape is {pred.shape} != (8)'

        return pred

In [ ]:
class PaperModel(nn.Module):
    def __init__(self, k, output_dim, hidden_dim):
        super(PaperModel, self).__init__()
        self.k = k
        self.embedding_dim = 100 + 4**k
        
        self.embedding = PaperEmbedding(k)
        self.convolution = Convolution(k)
        self.lstm_attn = LstmAttn( k, self.embedding_dim, output_dim)
        self.binary_classification_head = BinaryClassificationHead( k, 2*output_dim, hidden_dim)
        
    def forward(self, input_tensor):
        
#         assert(input_tensor.shape[1] == 101 - self.k + 1 ), f'X.shape {input_tensor.shape} != (_,100-self.k+1)'
#         print(input_tensor.shape)
        embedded_input = self.embedding(input_tensor)
#         print(embedded_input)
#         print(embedded_input.shape)
#         assert( embedded_input.shape == (8, 101 - self.k + 1, 100 + 4**self.k) )
        
        X = self.convolution(embedded_input)
        
#         print(X.shape)
        
        X = self.lstm_attn(X)
                
        X = self.binary_classification_head(X)
        
        return X
        
        
    
    


        

In [12]:
def evaluate_model(dataloader, model, loss_fn):

    sum_of_loss = 0
    correct = 0
    total = 0

    model.eval()
    for local_batch, local_labels in dataloader:
        input_tensor = get_tensor(local_batch, model.k)
        label_tensor = torch.tensor((np.array(local_labels)).astype(np.float32))
        predicted_output = model(input_tensor)
        # val_loss = loss_fn(predict_output, output)
        val_loss = loss_fn(predicted_output, label_tensor)

        sum_of_loss += val_loss.item()
        
        correct += (torch.round(predicted_output) == label_tensor).sum().item()
        total += len(local_labels)

    accuracy = (correct / total)
    loss = sum_of_loss/len(dataloader)

    print(loss)
    print(accuracy)
    
    return(loss, accuracy)


def train(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs):
    print('starting training')
    curr_epoch = 1
    for epoch in range(epochs):
        print(f'Starting epoch: {curr_epoch}/{epochs}')
        epoch_start_time = time.time()
        # Training
        model.train()
#         i = 1
#         batch_size = 64
#         num_batches = math.ceil(len(train_sets)/batch_size)
        for local_batch, local_labels in train_dataloader:
            print(local_batch)
            
#             print(f'Starting Run: {i}/{num_batches}')
#             i += 1
            input_tensor = get_tensor(local_batch, model.k)
            label_tensor = torch.tensor((np.array(local_labels)).astype(np.float32))
            optimizer.zero_grad()
            predicted_output = model(input_tensor)            
            curr_loss = loss_fn(predicted_output, label_tensor)
            curr_loss.backward()
            optimizer.step()
            
        evaluate_model(train_dataloader, model, loss_fn)
        
        print(f'Time for epoch {curr_epoch}: {time.time() - epoch_start_time}')
        curr_epoch += 1 
            
            
    print('finished training')

In [13]:
folders = list(summary_df['ID'])

folders_using = folders[0:1]
# folders_using = folders
print(folders_using)


# Training Generators
train_set_lst = []
for folder in folders_using:
    curr_dataset = DNADataset(folder, 'train')
    train_set_lst.append(curr_dataset)


train_sets = torch.utils.data.ConcatDataset(train_set_lst)
train_generator = torch.utils.data.DataLoader(train_sets, batch_size = 64)


# Test Generators
test_set_lst = []
for folder in folders_using:
    test_set_lst.append(DNADataset(folder, 'test'))
    
test_sets = torch.utils.data.ConcatDataset(test_set_lst)
test_generator = torch.utils.data.DataLoader(test_sets)

# Loss Function
ce_loss = torch.nn.BCELoss( reduction = 'mean' )

# Paper model
paper_model = PaperModel(k = 3, output_dim = 16, hidden_dim = 16)

#Optimizer
adam_optimizer = torch.optim.Adam(paper_model.parameters(), lr= 0.001)

train(paper_model, train_generator, test_generator, ce_loss, adam_optimizer, epochs = 10)

['wgEncodeAwgTfbsBroadDnd41CtcfUniPk']


NameError: name 'PaperModel' is not defined

In [ ]:
c = torch.tensor([[1,2],[3,4]])
c.shape == (2,2)

In [790]:
t = torch.tensor([1,2,3,4,5,6,7,8])
t.shape == (8,)

True

In [872]:
import math
math.ceil(1.1)

2

In [823]:
loss = nn.BCELoss( reduction='mean')
# input = torch.randn(3, requires_grad=True)
input = torch.tensor([0.4616, 0.4545, 0.4649, 0.4563, 0.4607, 0.4606, 0.4591, 0.4684])
target = torch.tensor([1, 1, 0, 1, 0, 1, 1, 0])
target = torch.tensor([1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0])
output = loss(input, target)
output


tensor(0.7218)